# BUILD A REDUCED ORDER MODEL

### Imports and custom class definitions

In [ ]:
import os
import io
import sys
import time
try:
    import mfem.par as mfem
except ModuleNotFoundError:
    msg = "PyMFEM is not installed yet. Install PyMFEM:\n"
    msg += "\tgit clone https://github.com/mfem/PyMFEM.git\n"
    msg += "\tcd PyMFEM\n"
    msg += "\tpython3 setup.py install --with-parallel\n"
    raise ModuleNotFoundError(msg)

from ctypes import c_double
from mfem.par import intArray
from os.path import expanduser, join, dirname
import numpy as np
from numpy import sin, cos, exp, sqrt, pi, abs, array, floor, log, sum

In [ ]:
from importlib import reload

In [ ]:
sys.path.append("../../build")
import pylibROM.linalg as libROM
from pylibROM.mfem import ComputeCtAB

In [ ]:
from stopwatch import StopWatch

#### By construct pylibROM is capable of handling parallel computations

In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
myid = comm.Get_rank()
num_procs = comm.Get_size()

#### Specifying arguments for MFEM

In [ ]:
ns = 8
train_freq_id = list(range(1, 9))


In [ ]:
from parser_config import get_parser
parser = get_parser()

# Merge phase:
args = parser.parse_args(f"-merge -ns {ns}".split())


In [ ]:
if (myid == 0):
    parser.print_options(args)

freq            = args.frequency
fom             = args.fom
offline         = args.offline
online          = args.online
merge           = args.merge
device_config   = args.device
id              = args.id
order           = args.order
nsets           = args.nset
coef            = args.coefficient
pa              = args.partial_assembly
static_cond     = args.static_condensation
visualization   = args.visualization
precision       = 8
paraview        = args.paraview

kappa = np.pi*freq

#### Enable hardware devices such as GPUs, and programming models such as CUDA, OCCA, RAJA and OpenMP based on command line options.

In [ ]:
device = mfem.Device(device_config)
if (myid == 0):
    device.Print()

### Load the data from FE models

#### Read the already generated (serial) mesh from the existing mesh file on all processors

In [ ]:
data_dir = "training_data/"

In [ ]:
if(merge):
    basisName = data_dir+"basis"
    basisFileName = "%s%d" % (basisName, id)
    mergeTimer = StopWatch()
    
    mesh_file = data_dir+"mesh.000000"
    mesh = mfem.Mesh(mesh_file, 0, 0) #No modifications
    dim = mesh.Dimension()

#### Define a parallel mesh by a partitioning of the serial mesh. Refine this mesh further in parallel to increase the resolution. Once the parallel mesh is defined, the serial mesh can be deleted.

In [ ]:
if(merge):
    pmesh = mfem.ParMesh(comm, mesh)
    mesh.Clear()

#### Define a parallel finite element space on the parallel mesh. Here we use continuous Lagrange finite elements of the specified order. If order < 1, we instead use an isoparametric/isogeometric space.

In [ ]:
if(merge):
    if (order > 0):
        fec = mfem.H1_FECollection(order, dim)
        delete_fec = True
    elif (pmesh.GetNodes()):
        fec = pmesh.GetNodes().OwnFEC()
        delete_fec = False
        if (myid == 0):
            print(f"Using isoparametric FEs: {fec.Name()}")
    else:
        fec = mfem.H1_FECollection(1, dim)
        delete_fec = True
    
    fespace = mfem.ParFiniteElementSpace(pmesh, fec)
    size = fespace.GlobalTrueVSize()
    if (myid == 0):
        print(f"Number of finite element unknowns: {size}")

### Merge data from FE models

In [ ]:
if (merge):
    max_num_snapshots = 100
    update_right_SV = False
    isIncremental = False
    
    mergeTimer.Start()
    options = libROM.Options(fespace.GetTrueVSize(), max_num_snapshots, 1,
                            update_right_SV)
    generator = libROM.BasisGenerator(options, isIncremental, basisName)
    for paramID in train_freq_id:
        snapshot_filename = "%s%d_snapshot" % (basisName, paramID)
        generator.loadSamples(snapshot_filename,"snapshot", 5)

    generator.endSamples() # save the merged basis file
    mergeTimer.Stop()
    if (myid == 0):
        print("Elapsed time for merging and building ROM basis: %e second\n" %
               mergeTimer.duration)
    del generator
    del options
    np.save('rom_data/merge_time',mergeTimer.duration)
    MPI.Finalize()
